In [1]:
import pandas as pd

faq_df = pd.read_json("../data/full_faq.json")
faq_df.head()

,question,answer,category
0,Di mana ada lokasi Rumah Sakit Siloam?,Ada 40 Rumah Sakit modern yang terdiri dari 13...,FAQ Website
1,Apa jenis kamar rawat inap yang tersedia?,Siloam Hospitals menawarkan banyak pilihan jen...,FAQ Website
2,Apa standar keamanan suplai darah yang diambil...,Untuk memastikan keamanan dan kualitas suplai ...,FAQ Website
3,Fasilitas apa yang dimiliki pusat keunggulan R...,Kardiologi: Di Cardiac Center kami terdapat Un...,FAQ Website
4,Apa standar kualitas yang dipatuhi oleh Siloam...,Siloam Hospitals mematuhi standar kualitas dar...,FAQ Website


In [2]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer(
    "intfloat/multilingual-e5-large",
    trust_remote_code=True
)

/workspaces/dev/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
question_embeddings = model.encode(
    faq_df["question"].tolist()
)

answer_embeddings = model.encode(
    faq_df["answer"].tolist()
)

combined_texts = [
    f"{q}\n{a}" for q, a in zip(faq_df["question"], faq_df["answer"])
]

combined_embeddings = model.encode(combined_texts)

print("Question embeddings shape:", question_embeddings.shape)
print("Answer embeddings shape:", answer_embeddings.shape)
print("Combined embeddings shape:", combined_embeddings.shape)


Question embeddings shape: (349, 1024)
Answer embeddings shape: (349, 1024)
Combined embeddings shape: (349, 1024)


In [5]:
question_embeddings

array([[ 0.00825853, -0.00684039, -0.01008988, ..., -0.04223974,
        -0.02853844,  0.01105917],
       [ 0.01679304,  0.00366306, -0.02188492, ..., -0.0539857 ,
        -0.02692915,  0.01102671],
       [ 0.0082514 , -0.02746587, -0.03842525, ..., -0.039239  ,
        -0.00496398,  0.02464821],
       ...,
       [-0.01962173, -0.01879275, -0.02104165, ..., -0.02994614,
        -0.01923648,  0.04575821],
       [-0.00625097,  0.00917968, -0.00012745, ..., -0.01775001,
        -0.05250166,  0.03142332],
       [-0.0138129 ,  0.00152469, -0.0200156 , ..., -0.02324898,
        -0.02810773,  0.04096591]], dtype=float32)

In [6]:
from qdrant_client import QdrantClient, models

dimensions = 1024
distance = models.Distance.COSINE

client = QdrantClient("http://localhost:6333")
# client.delete_collection("faq")
client.create_collection(
    collection_name="faq",
    vectors_config={
        "question": models.VectorParams(
            size=dimensions,
            distance=distance,
        ),
        "answer": models.VectorParams(
            size=dimensions,
            distance=distance,
        ),
        "combined": models.VectorParams(
            size=dimensions,
            distance=distance,
        ),
    },
    optimizers_config=models.OptimizersConfigDiff(
        default_segment_number=2,
        indexing_threshold=100,
    ),
)


True

In [7]:
client.upload_collection(
    collection_name="faq",
    vectors={
        "question": question_embeddings,
        "answer": answer_embeddings,
        "combined": combined_embeddings
    },
    payload=faq_df.to_dict(orient="records"),
    ids=faq_df.index.tolist(),
    batch_size=64,
)


In [8]:
client.count("faq")

CountResult(count=349)

In [9]:
import time

time.sleep(1.0)
collection = client.get_collection("faq")
while collection.status != models.CollectionStatus.GREEN:
    time.sleep(1.0)
    collection = client.get_collection("faq")
    
collection


CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=1047, points_count=349, segments_count=2, config=CollectionConfig(params=CollectionParams(vectors={'answer': VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), 'combined': VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), 'question': VectorParams(size=1024, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None)}, shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, m

In [ ]:
import pandas as pd
from collections import defaultdict
from ranx import Qrels, Run, compare

# 1. Prepare Qrels (Ground Truth)
# -------------------------------
# Each question should retrieve its own FAQ entry
qrels_data = [{
    "query_id": f"q_{idx}",
    "doc_id": str(idx),
    "score": 10  # Exact match
} for idx in faq_df.index]

qrels = Qrels.from_df(
    pd.DataFrame(qrels_data),
    q_id_col="query_id",
    doc_id_col="doc_id",
    score_col="score",
)

# 2. Encode Questions
# -------------------
question_embeddings = model.encode(faq_df["question"].tolist())

# 3. Create Search Runs
# ---------------------
question_run_dict = defaultdict(dict)
answer_run_dict = defaultdict(dict)
combined_run_dict = defaultdict(dict)

for idx, (q_emb, row) in enumerate(zip(question_embeddings, faq_df.iterrows())):
    query_id = f"q_{idx}"
    
    # Search against QUESTION vectors
    question_results = client.search(
        collection_name="faq",
        query_vector=models.NamedVector(
            name="question",
            vector=q_emb.tolist()
        ),
        limit=20,
    )
    for hit in question_results:
        question_run_dict[query_id][str(hit.id)] = hit.score
    
    # Search against ANSWER vectors
    answer_results = client.search(
        collection_name="faq",
        query_vector=models.NamedVector(
            name="answer",
            vector=q_emb.tolist()
        ),
        limit=20,
    )
    for hit in answer_results:
        answer_run_dict[query_id][str(hit.id)] = hit.score

    # Search against QUESTION ANSWER vectors
    results = client.search(
        collection_name="faq",
        query_vector=models.NamedVector(
            name="combined", 
            vector=q_emb.tolist()
        ),
        limit=20,
    )
    for hit in results:
        combined_run_dict[query_id][str(hit.id)] = hit.score

# 4. Create Run Objects
# ---------------------
question_run = Run(question_run_dict, name="question_vectors")
answer_run = Run(answer_run_dict, name="answer_vectors")
combined_run = Run(combined_run_dict, name="question_answer_vector")

# 5. Compare at k=20
# ------------------
report = compare(
    qrels=qrels,
    runs=[question_run, answer_run, combined_run],
    metrics=["precision@20", "recall@20", "mrr@20", "ndcg@20"],
)

print(report)

/tmp/ipykernel_24812/3062935410.py:35: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  question_results = client.search(
/tmp/ipykernel_24812/3062935410.py:50: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  answer_results = client.search(
/tmp/ipykernel_24812/3062935410.py:65: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


#    Model                   P@20    Recall@20    MRR@20    NDCG@20
---  ----------------------  ------  -----------  --------  ---------
a    question_vectors        0.050ᵇ  1.000ᵇ       0.994ᵇ    0.996ᵇ
b    answer_vectors          0.038   0.751        0.625     0.654
c    question_answer_vector  0.050ᵇ  1.000ᵇ       0.981ᵇ    0.986ᵇ


In [12]:
test_list = [
    { "query": "siloam ada dimana saja", "expected": "Di mana ada lokasi Rumah Sakit Siloam?" },
    { "query": "jam buka mcu", "expected": "Jam buka MCU / Medical Check Up?" },
    { "query": "layanan kesehatan di rumah", "expected": "Apa itu Siloam at Home? Apakah Siloam memberikan layanan kesehatan di rumah atau Homecare?" },
    { "query": "cek obat beli di siloam dari aplikasi", "expected": "Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?" },
    { "query": "asuransi kerjasa sama dengan siloam", "expected": "Asuransi apa saja yang bekerja sama dengan Siloam?" },
    { "query": "jenis kamar rawat inap apa saja", "expected": "Apa jenis kamar rawat inap yang tersedia?" },
    { "query": "standar keamanan darah di rumah sakit seperti apa", "expected": "Apa standar keamanan suplai darah yang diambil di rumah sakit?" },
    { "query": "fasilitas pusat keunggulan rumah sakit apa saja", "expected": "Fasilitas apa yang dimiliki pusat keunggulan Rumah Sakit?" },
    { "query": "standar kualitas yang diterapkan siloam hospitals", "expected": "Apa standar kualitas yang dipatuhi oleh Siloam Hospitals?" },
    { "query": "pembayaran apa saja tersedia di siloam hospitals", "expected": "Metode pembayaran apa yang tersedia di Siloam Hospitals?" },
    { "query": "kamar rumah sakit menyediakan fasilitas apa saja", "expected": "Fasilitas apa yang disediakan di dalam kamar?" },
    { "query": "apakah merokok diperbolehkan di rumah sakit", "expected": "Apakah merokok diperbolehkan di area rumah sakit?" },
    { "query": "berapa banyak pasien di siloam hospitals tiap tahun", "expected": "Berapa banyak pasien yang dirawat Siloam Hospitals setiap tahun?" },
    { "query": "apakah siloam hospitals memberikan perkiraan biaya perawatan", "expected": "Apakah rumah sakit Siloam menawarkan perkiraan biaya perawatan?" },
    { "query": "asuransi kesehatan diterima di siloam hospitals", "expected": "Jenis asuransi kesehatan apa saja yang diterima di Siloam Hospitals?" },
    { "query": "bagaimana jika dokter pilihan tidak ada", "expected": "Bagaimana jika dokter pilihan saya tidak tersedia?" },
    { "query": "cara berkomunikasi dengan dokter bagi yang tidak fasih bahasa indonesia", "expected": "Saya tidak fasih berbahasa Indonesia. Bagaimana cara berkomunikasi dengan dokter?" },
    { "query": "barang apa yang harus dibawa ke siloam hospital", "expected": "Barang apa saja yang perlu saya bawa ke Siloam Hospital?" },
    { "query": "apakah perlu janji untuk spesialis", "expected": "Apakah saya perlu membuat janji untuk bertemu dengan spesialis?" },
    { "query": "jam kunjungan pasien di rumah sakit", "expected": "Pada jam berapa pengunjung diperbolehkan untuk mengunjungi pasien?" },
    { "query": "bolehkah kasih bunga atau hadiah ke pasien", "expected": "Apakah saya boleh memberikan bunga atau hadiah kepada pasien di Siloam Hospital?" },
    { "query": "cara hubungi pasien di ICU", "expected": "Bagaimana saya bisa menghubungi pasien di Unit Perawatan Intensif?" },
    { "query": "ada daftar hotel di sekitar siloam hospitals", "expected": "Apakah Anda memiliki daftar hotel dekat Rumah Sakit Siloam?" },
    { "query": "tamu boleh kunjungan setelah jam kerja tidak", "expected": "Apakah tamu diizinkan untuk berkunjung setelah jam kerja?" },
    { "query": "anak-anak boleh kunjungi pasien di rumah sakit", "expected": "Apakah anak-anak diperbolehkan mengunjungi pasien di Siloam Hospitals?" },
    { "query": "dokumen apa yang harus dibawa untuk ambil medical report", "expected": "Dokumen apa saja yang harus saya bawa saat ingin mengambil medical report/formulir asuransi/dokumen medis?" },
    { "query": "cara minta informasi medis", "expected": "Bagaimana cara saya meminta informasi medis saya?" },
    { "query": "cara dapat salinan rekam medis", "expected": "Bagaimana saya bisa mendapatkan salinan rekam medis saya untuk diri saya sendiri atau untuk dikirimkan kepada orang lain selain diri saya sendiri, seperti dokter, pengacara, perusahaan asuransi atau orang lain?" },
    { "query": "jam operasional mcu", "expected": "Jam buka MCU / Medical Check Up?" },
    { "query": "apakah perlu puasa sebelum mcu", "expected": "Apakah MCU / Medical Check Up perlu melakukan puasa?" },
    { "query": "berapa lama waktu untuk mcu test", "expected": "Berapa lama waktu yang dibutuhkan untuk MCU / Medical Check Up tes?" },
    { "query": "paket mcu rekomendasi apa yang terlengkap", "expected": "Apa rekomendasi paket MCU / Medical Check Up terlengkap saat ini?" },
    { "query": "apakah semua siloam hospitals sediakan mcu", "expected": "Apakah saya bisa melakukan MCU / Medical Check Up di semua Siloam?" },
    { "query": "cara daftar mcu", "expected": "Bagaimana cara mendaftar MCU / Medical Check Up?" },
    { "query": "apa yang harus dibawa untuk mcu", "expected": "Apa yang perlu dibawa ketika melakukan MCU / Medical Check Up?" },
    { "query": "bedanya daftar mcu lewat website dan telepon", "expected": "Apa bedanya mendaftar MCU / Medical Check Up lewat website dengan lewat telepon/datang langsung?" },
    { "query": "metode pembayaran untuk mcu", "expected": "Apa saja metode pembayaran pada pendaftaran MCU / Medical Check Up?" },
    { "query": "hasil mcu yang didapatkan", "expected": "Hasil apa yang saya dapatkan setelah tes MCU / Medical Check Up di Siloam?" },
    { "query": "berapa lama hasil mcu keluar", "expected": "Berapa lama hasil dari MCU / Medical Check Up tes?" },
    { "query": "persiapan sebelum datang ke mcu", "expected": "Apa saja yang harus saya lakukan sebelum datang ke MCU / Medical Check Up (persiapan yang perlu saya lakukan)?" },
    { "query": "siloam at home itu apa", "expected": "Apa itu Siloam at Home? Apakah Siloam memberikan layanan kesehatan di rumah atau Homecare?" },
    { "query": "layanan siloam at home apa saja", "expected": "Layanan Siloam at Home apa saja yang diberikan oleh Siloam?" },
    { "query": "cara pesan layanan siloam at home", "expected": "Bagaimana cara melakukan pemesanan layanan Siloam at Home?" },
    { "query": "metode pembayaran untuk siloam at home", "expected": "Apa saja metode pembayaran yang saya bisa gunakan untuk Siloam at Home?" },
    { "query": "apakah siloam at home sediakan perawat di rumah", "expected": "Apakah Siloam at Home memfasilitasi layanan perawat tinggal di rumah pasien?" },
    { "query": "cek obat dari aplikasi siloam", "expected": "Halo saya mau cek obat yang saya beli dari Siloam, bisakah dari aplikasi MySiloam?" },
    { "query": "cara pesan dokter lewat aplikasi mysiloam", "expected": "Bagaimana cara pesan dokter dari aplikasi MySiloam?" },
    { "query": "jika booking lewat aplikasi, apakah slot sesuai jadwal dokter", "expected": "Jika saya udah booking dari aplikasi MySiloam, apakah saya pasti terdaftar slot yang sesuai dengan jadwal dokter?" },
    { "query": "sudah booking di mysiloam, apakah saya terdaftar", "expected": "Saya sudah booking di aplikasi MySiloam, apakah saya sudah terdaftar?" },
    { "query": "fitur apa saja di aplikasi mysiloam", "expected": "Fitur apa sajakah yang ada di aplikasi MySiloam?" },
    { "query": "cara download aplikasi mysiloam", "expected": "Bagaimana cara download aplikasi MySiloam?" },
    { "query": "apa itu menu rekam medis di mysiloam", "expected": "Apa itu menu rekam medis pada aplikasi MySiloam?" },
    { "query": "cara aktivasi rekam medis", "expected": "Gimana caranya untuk buka atau aktivasi rekam medis?" },
    { "query": "data apa saja ada di menu rekam medis", "expected": "Data apa saja yang tersedia di menu rekam medis di aplikasi MySiloam?" },
    { "query": "data rawat inap yang ditampilkan apa saja", "expected": "Data apa saja yang tersedia untuk rawat inap?" },
    { "query": "asuransi yang kerjasama dengan siloam apa saja", "expected": "Asuransi apa saja yang bekerja sama dengan Siloam?" },
    { "query": "jika asuransi tidak kerjasama, apa yang harus dilakukan", "expected": "Apa yang harus dilakukan jika asuransi tidak bekerja sama dengan rumah sakit?" },
    { "query": "cara bayar dengan asuransi", "expected": "Bagaimana alur dalam membayar dengan asuransi?" },
    { "query": "berapa lama urusan asuransi sebelum konsultasi dokter", "expected": "Berapa lama waktu untuk pasien untuk mengurus asuransi sebelum konsultasi dengan dokter?" },
    { "query": "apa yang harus disiapkan saat daftar dengan asuransi", "expected": "Apa saja yang harus dipersiapkan saat mendaftar ke rumah sakit menggunakan asuransi?" },
    { "query": "data apa yang diperlukan saat pakai bpjs", "expected": "Data apa yang dibutuhkan saat menggunakan BPJS di Siloam Hospitals?" },
    { "query": "apakah kondisi darurat perlu surat rujukan", "expected": "Apakah kondisi gawat darurat memerlukan Surat Rujukan dari Puskesmas/Klinik seperti mendaftar rawat jalan?" },
    { "query": "rumah sakit siloam mana aja terima bpjs", "expected": "Rumah Sakit Siloam mana saja yang menerima pasien BPJS?" },
    { "query": "bisakah daftar bpjs di hari h", "expected": "Apakah saya bisa daftar BPJS di hari H?" },
    { "query": "jam operasional konsultasi bpjs", "expected": "Jam operasional untuk konsultasi BPJS?" },
    { "query": "cara reschedule appointment bpjs", "expected": "Bagaimana cara melakukan reschedule appointment apabila saya sudah melakukan pendaftaran via website?" },
    { "query": "masa berlaku surat rujukan bpjs", "expected": "Berapa lama masa berlaku Surat Rujukan yang diterbitkan Puskesmas/Klinik untuk berobat ke Rumah Sakit?" },
    { "query": "cara buat appointment bpjs", "expected": "Bagaimana cara membuat appointment/ mendaftar untuk antrian untuk BPJS?" },
    { "query": "cara dapatkan surat rujukan siloam", "expected": "Bagaimana cara mendapatkan surat rujukan untuk diperiksa oleh RS Siloam?" },
    { "query": "dimana beli produk laboratorium online", "expected": "Dimana saya dapat membeli produk laboratorium / radiologi secara online?" },
    { "query": "produk yang dicari tidak ada di website", "expected": "Produk yang saya cari tidak ada di website" },
    { "query": "cek ketersediaan prosedur di rumah sakit", "expected": "Ketersediaan prosedur pada rumah sakit" },
    { "query": "perlukah surat rujukan dokter untuk pembelian", "expected": "Apakah memerlukan surat rujukan dari dokter?" },
    { "query": "apakah jadwal di website sudah sesuai", "expected": "Apakah jadwal yang ditampilkan sudah sesuai?" },
    { "query": "biaya sudah termasuk konsultasi atau tidak", "expected": "Apakah biaya yang dibayarkan sudah termasuk dengan biaya konsultasi?" },
    { "query": "cara atur ulang jadwal", "expected": "Bagaimana jika saya ingin mengatur jadwal ulang?" },
    { "query": "ada batas atur ulang jadwal?", "expected": "Apakah ada limitasi untuk mengatur jadwal ulang?" },
    { "query": "jika butuh layanan homecare, bagaimana caranya", "expected": "Bagaimana jika saya butuh layanan homecare?" },
    { "query": "apakah bisa pesan homeservice dari semua siloam hospitals", "expected": "Apakah saya bisa melakukan pemesanan homeservice dari semua rumah sakit Siloam?" },
    { "query": "cara minta pengembalian dana", "expected": "Bagaimana jika saya ingin pengembalian dana?" },
    { "query": "pendaftaran radiologi di hari h boleh gak", "expected": "Apakah pendaftaran radiologi dapat dilakukan pada hari H?" },
    { "query": "jam operasional radiologi emergency", "expected": "Jam operasional radiologi?" },
    { "query": "jam operasional laboratorium untuk non-emergency", "expected": "Jam operasional laboratorium?" },
    { "query": "lokasi radiologi atau laboratorium di rumah sakit", "expected": "Dimanakah lokasi radiologi/laboratorium?" },
    { "query": "cara pembayaran untuk laboratorium", "expected": "Cara pembayaran?" },
    { "query": "cara pemesanan laboratorium", "expected": "Cara pemesanan?" },
    { "query": "bagaimana memasukkan diskon di pemesanan laboratorium", "expected": "Bagaimana cara memasukan diskon pada pemesanan laboratorium dan radiologi di Website?" },
    { "query": "cara pesan telekonsultasi", "expected": "Bagaimana cara melakukan pemesanan Telekonsultasi?" },
    { "query": "prosedur telekonsultasi seperti apa", "expected": "Bagaimana prosedur melakukan Telekonsultasi?" },
    { "query": "setelah pesan telekonsultasi, apa yang didapat", "expected": "Apa yang akan didapatkan setelah melakukan pemesanan Telekonsultasi?" },
    { "query": "dimana dapatkan link telekonsultasi", "expected": "Dimana saya bisa mendapatkan link Telekonsultasi?" },
    { "query": "berapa lama durasi telekonsultasi", "expected": "Berapa lama durasi Telekonsultasi?" },
    { "query": "dokter bisa apa saja di telekonsultasi", "expected": "Apa saja yang Dokter bisa lakukan dalam Teleconsultasi?" },
    { "query": "jika salah pesan telekonsultasi, bagaimana caranya", "expected": "Bagaimana jika saya salah dalam melakukan pemesanan telekonsultasi?" },
    { "query": "prosedur refund telekonsultasi", "expected": "Bagaimana prosedur untuk pengembalian dana yang sudah saya bayarkan pada pemesanan Telekonsultasi?" },
    { "query": "bisa atur ulang jadwal telekonsultasi di hari h gak", "expected": "Apakah bisa melakukan atur ulang jadwal Telekonsultasi di hari H?" },
    { "query": "jika dokter tidak hadir di telekonsultasi, apa yang dilakukan", "expected": "Bagaimana jika Dokter berhalangan untuk hadir ketika Teleconsultation saya?" },
    { "query": "setelah telekonsultasi, bagaimana alur resep obat", "expected": "Bagaimana alur jika saya di resepkan obat oleh Dokter setelah selesai melakukan Telekonsultasi?" },
    { "query": "jika ada rujukan untuk lab dan radiology, apa yang harus dilakukan", "expected": "Bagaimana jika ada rujukan untuk pemeriksaan Lab dan Radiology?" },
    { "query": "cara pesan telechat", "expected": "Bagaimana cara melakukan pemesanan Telechat?" }
]


In [ ]:
# 1. Siapkan Qrels (Ground Truth)
# -------------------------------
# Cari index pertanyaan yang diharapkan di faq_df
expected_indices = []
for test_case in test_list:
    expected_question = test_case["expected"]
    idx = faq_df[faq_df["question"] == expected_question].index[0]
    expected_indices.append(idx)

# Buat DataFrame Qrels
qrels_data = [{
    "query_id": f"test_{i}",
    "doc_id": str(idx),
    "score": 10
} for i, idx in enumerate(expected_indices)]

qrels = Qrels.from_df(
    pd.DataFrame(qrels_data),
    q_id_col="query_id",
    doc_id_col="doc_id",
    score_col="score",
)

# 2. Encode Test Queries
# ----------------------
test_queries = [case["query"] for case in test_list]
test_embeddings = model.encode(test_queries)

# 3. Lakukan Pencarian untuk Setiap Vektor
# ----------------------------------------
question_run_dict = defaultdict(dict)
answer_run_dict = defaultdict(dict)
combined_run_dict = defaultdict(dict)

for i, (query, query_emb) in enumerate(zip(test_queries, test_embeddings)):
    query_id = f"test_{i}"
    
    # Search ke question vector
    results = client.search(
        collection_name="faq",
        query_vector=models.NamedVector(name="question", vector=query_emb.tolist()),
        limit=20,
    )
    for hit in results:
        question_run_dict[query_id][str(hit.id)] = hit.score
    
    # Search ke answer vector
    results = client.search(
        collection_name="faq",
        query_vector=models.NamedVector(name="answer", vector=query_emb.tolist()),
        limit=20,
    )
    for hit in results:
        answer_run_dict[query_id][str(hit.id)] = hit.score
    
    # Search ke question_answer vector
    results = client.search(
        collection_name="faq",
        query_vector=models.NamedVector(name="combined", vector=query_emb.tolist()),
        limit=20,
    )
    for hit in results:
        combined_run_dict[query_id][str(hit.id)] = hit.score

# 4. Buat Run dan Bandingkan
# --------------------------
question_run = Run(question_run_dict, name="question_vector")
answer_run = Run(answer_run_dict, name="answer_vector")
combined_run = Run(combined_run_dict, name="combined_vector")

report = compare(
    qrels=qrels,
    runs=[question_run, answer_run, combined_run],
    metrics=["precision@20", "recall@20", "mrr@20", "ndcg@20"],
)

print(report)

/tmp/ipykernel_24812/1778145601.py:39: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
/tmp/ipykernel_24812/1778145601.py:51: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(
/tmp/ipykernel_24812/1778145601.py:63: DeprecationWarning: `search` method is deprecated and will be removed in the future. Use `query_points` instead.
  results = client.search(


#    Model            P@20    Recall@20    MRR@20    NDCG@20
---  ---------------  ------  -----------  --------  ---------
a    question_vector  0.050ᵇ  1.000ᵇ       0.967ᵇ    0.975ᵇ
b    answer_vector    0.041   0.830        0.650     0.693
c    combined_vector  0.049ᵇ  0.990ᵇ       0.955ᵇ    0.964ᵇ
